In [22]:
! pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [23]:
import pandas as pd  
import numpy as np
import re
import random
from unidecode import unidecode 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys  
# Nuevas importaciones:
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from time import sleep  

In [24]:
def extraccion_detalles_imdb (archivo_csv = 'Ruta o nombre del archivo.csv', columna1 = 'Nombre columna títulos', columna2 = 'Nombre columna años' , salida_detalles = "Ruta o nombre de salida_detalles.csv" , salida_actores = "Ruta o nombre de salida_actores.csv"):   
    
    #! ------------------------------------------------------------------------------ Abre el CSV
    # Cargar el archivo CSV
    peliculas_df = pd.read_csv(archivo_csv)

    
    #! ------------------------------------------------------------------------------ Empieza la extracción con Selenium
    
    # Iniciar el navegador Chrome
    options = webdriver.ChromeOptions()
    options.add_argument("--accept-lang=es")
    driver = webdriver.Chrome(options=options)

    # Abrir la página de IMDb
    driver.get("https://www.imdb.com/")
    driver.maximize_window()
    sleep(3)

    try:
        # Aceptar las cookies si es necesario
        driver.find_element("css selector" , "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW").click()
    except:
        pass

    sleep(random.randint(1, 3))

    print("--- Empieza la extracción de datos ---")
    print(" ")

    lista_detalles_peliculas = []
    lista_actores = []
    ultimo_descanso_corto = 0
    ultimo_descanso_largo = 0

    for indice, pelicula in peliculas_df.iterrows():
        
        # Condición para asegurar que una vez que se haya hecho un descanso corto, no se repita hasta pasado un mínimo de iteraciones
        if indice >= ultimo_descanso_corto + random.randint(100 , 129):
            descanso_corto = random.randint(3, 15) # Escoge un número random y lo asigna a la variable descanso corto
        
            if indice != 0 and indice % descanso_corto == 0:
                ultimo_descanso_corto = indice 
                print(f"Hago un pequeño break. Estoy en la iteración número {indice}.")
                print(f"Volveré en unos 5-10 segundos")
                sleep(random.randint(5 , 10) + random.uniform(0.1 , 0.5))
                print(f"¡Ya estoy de vuelta! Sigo con lo mío")
                print(' ')
            
        # Condición para asegurar que una vez que se haya hecho un descanso largo, no se repita hasta pasado un mínimo de iteraciones
        if indice >= ultimo_descanso_largo + random.randint(530 , 550):
            descanso_largo = random.randint(3, 12) # Escoge un número random y lo asigna a la variable descanso largo
            
            # Si el número de iteración es divisible por el número de descanso, se para durante una cantidad de segundos aleatoria
            if indice != 0 and indice % descanso_largo == 0:
                tiempo_descanso = random.randint(10 , 60)
                ultimo_descanso_largo = indice 
                print(f"Paro a por un café ☕. Estoy en la iteración número {indice}.")
                print(f"Volveré en {round(tiempo_descanso/60 , 1)} minutos")
                sleep(tiempo_descanso)
                print(f"¡Ya estoy de vuelta! Sigo con lo mío")
                print(' ')
        
        nombre_peli = pelicula[columna1].lower()  # Obtener el nombre de la película
        anio_peli = int(pelicula[columna2])  # Obtener el año de la película
        string_buscar = f"{nombre_peli}, {anio_peli}"  # Definir la frase de búsqueda
        encontrada = False
        
        # Establecemos variables de inicio
        puntuacion_encontrada = False
        reparto_encontrado = False
        duracion_encontrada = False
        argumento_encontrado = False
        guion_encontrado = False
        direccion_encontrada = False
        datos_reparto = "-"
        
        # Si encuentra una página con error 404 al inicio de cada iteración del bucle, vuelve a la home
        try:
            try: 
                driver.find_element('css selector', '#error > div.error_attrib > a').click()
                sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5))
                print(f"¡Vaya! Me han pillado, pero no pasa nada: Acabo de sortear el error 404 🥷🏻")
                print(" ")
            except:
                driver.find_element('css selector', '#error > div.error_message > a').click()
                sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5)) 
                print(f"¡Vaya! Me han pillado, pero no pasa nada: Acabo de sortear el error 404 🥷🏻")
                print(" ")
        except:
            pass

        # Intenta realizar la búsqueda en IMDb
        try:  
            driver.find_element("xpath", '//*[@id="suggestion-search"]').clear()
            driver.find_element("xpath", '//*[@id="suggestion-search"]').send_keys(string_buscar, Keys.ENTER)
            sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5))
            
            try:
                'buscar' in driver.find_element("xpath", '//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[1]/h1').text.lower()
    
            except:
                driver.find_element("xpath", '//*[@id="suggestion-search-button"]').click()
                sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5))
                
        # Si encuentra una página con error 404 al acceder a la barra de búsqueda, vuelve a la home                                                                 
        except:
            print("No se encuentra la barra de búsqueda")
            print(" ")
            sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5))
            try:
                try: 
                    driver.find_element('css selector', '#error > div.error_attrib > a').click()
                    sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5))
                    print(f"¡Vaya! Me han pillado, pero no pasa nada: Acabo de sortear el error 404 🥷🏻")
                    print(" ")
                except:
                    driver.find_element('css selector', '#error > div.error_message > a').click()
                    sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5)) 
                    print(f"¡Vaya! Me han pillado, pero no pasa nada: Acabo de sortear el error 404 🥷🏻")
                    print(" ")
            except:
                pass
        
        # Itera sobre los resultados de la búsqueda
        for i in range(1, 6):
            try:
                
                try: 
                    espera = 10 #! Si da un error, probar a aumentar el tiempo de espera a más segundos
                    # Obtiene el título y el año del resultado actual
                    datos_resultado = WebDriverWait(driver , espera).until(EC.presence_of_element_located((By.XPATH , f'//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[{i}]/div[2]')))
                    datos_resultado = datos_resultado.text.lower()
                
                except:
                    pass
                
                titulo_resultado = datos_resultado.split('\n')[0].strip()
                anio_resultado = int(re.findall('\d{4}' , datos_resultado.split('\n')[1].strip())[0])
                                
            except:
                print("Error al obtener datos del resultado")
                print(" ")
                
                # Si encuentra una página con error 404 al acceder a los resultados de búsqueda, vuelve a la home 
                try:
                    try: 
                        driver.find_element('css selector', '#error > div.error_attrib > a').click()
                        sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5))
                        print(f"¡Vaya! Me han pillado, pero no pasa nada: Acabo de sortear el error 404 🥷🏻")
                        print(" ")
                    except:
                        driver.find_element('css selector', '#error > div.error_message > a').click()
                        sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5))
                        print(f"¡Vaya! Me han pillado, pero no pasa nada: Acabo de sortear el error 404 🥷🏻")
                        print(" ") 
                except:
                    break
            
            if unidecode(nombre_peli) == unidecode(titulo_resultado) and anio_peli == anio_resultado:
                
                try:
                    
                    espera = 10 #! Si da un error, probar a aumentar el tiempo de espera a más segundos
                    # Hace click en el resultado donde se encuentran las coincidencias de título y año
                    resultado_equivalente = WebDriverWait(driver , espera).until(EC.element_to_be_clickable((By.XPATH ,f'//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[{i}]/div[1]/div')))
                    resultado_equivalente.click()
                    
                except:
                    break
        
                sleep(random.randint(3 , 4) + random.uniform(0.1 , 0.5)) 
                
                try: 
                    info = driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text 
                    
                except: 
                    try:
                        try: 
                            driver.find_element('css selector', '#error > div.error_attrib > a').click()
                            sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5))
                        except:
                            driver.find_element('css selector', '#error > div.error_message > a').click()
                            sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5)) 
                    except:
                        break
                
                info_separada = info.split('\n')
                
                # Itera por los elementos de la cabecera
                for indice , elemento in enumerate(info_separada):
                    
                    patron_tiempo = r'(\d{1,2}h\s)?\d{1,2}min'
                    patron_anio = r'\d{4}'
                    
                    if 'direccion' == unidecode(elemento.lower()):
                        datos_direccion = info_separada[indice+1]
                        direccion_encontrada = True
                        
                        # Extrae todas las coincidencias de nombre y apellidos de la string
                        totales_d = re.findall('[A-Z][a-z]*', datos_direccion)
                        
                        # Si hay más de 1 nombre y 1 apellido, entonces guarda en una lista la combinación de todos ellos
                        if len(totales_d) > 2 and len(totales_d) % 2 == 0:
                            datos_direccion = [f"{totales_d[i]} {totales_d[i+1]}" for i in range(0, len(totales_d), 2)]  
                            
                        elif len(totales_d) > 2 and len(totales_d) % 2 != 0:
                            # Si la lista de nombres y apellidos es impar, combina todos los elementos excepto el último y añade el último al penúltimo
                            datos_direccion = [f"{totales_d[i]} {totales_d[i+1]}" for i in range(0, len(totales_d) - 2, 2)]
                            
                            # Agrega el último elemento al penúltimo
                            datos_direccion[-1] += f" {totales_d[-1]}"                       
                    
                    if 'guion' == unidecode(elemento.lower()):
                        datos_guion = info_separada[indice+1]
                        guion_encontrado = True
                        
                        # Extrae todas las coincidencias de nombre y apellidos de la string
                        totales_g = re.findall('[A-Z][a-z]*', datos_guion)
                        
                        # Si hay más de 1 nombre y 1 apellido, entonces guarda en una lista la combinación de todos ellos
                        if len(totales_g) > 2 and len(totales_g) % 2 == 0:
                            datos_guion = [f"{totales_g[i]} {totales_g[i+1]}" for i in range(0, len(totales_g), 2)]
                        
                        elif len(totales_g) > 2 and len(totales_g) % 2 != 0:
                            # Si la lista de nombres y apellidos es impar, combina todos los elementos excepto el último y añade el último al penúltimo
                            datos_guion = [f"{totales_g[i]} {totales_g[i+1]}" for i in range(0, len(totales_g) - 2, 2)]
                            
                            # Agrega el último elemento al penúltimo
                            datos_guion[-1] += f" {totales_g[-1]}"
                        
                    if 'argumento' in elemento.lower() and unidecode(info_separada[indice+1].lower()) != 'direccion' and unidecode(info_separada[indice+1].lower()) != 'guion':
                        argumento = info_separada[indice+1]
                        argumento_encontrado = True
                        
                    if 'puntuacion en imdb' == unidecode(elemento.lower()):
                        puntuacion = info_separada[indice+1]
                        puntuacion = float(puntuacion.replace(',' , '.'))
                        puntuacion_encontrada = True
                    
                    if 'reparto' in elemento.lower():
                        reparto_encontrado = True
                        
                    if re.match(patron_tiempo, elemento) and re.match(patron_anio , info_separada[indice-2]):
                        horas = info_separada[indice]   
                        duracion_encontrada = True
                                    
                        if len(horas.split()) > 1:
                            duracion_minutos = int(horas.split()[0].replace('h' , ''))*60 + int(horas.split()[1].replace('min', ''))
                        
                        else:
                            duracion_minutos = int(horas.replace('min', '')) 
                
                # Si no había datos de puntuación, asigna '-' por defecto            
                if not puntuacion_encontrada:
                    puntuacion = '-'
                
                # Si no había datos de duración, asigna '-' por defecto     
                if not duracion_encontrada:
                    duracion_minutos = '-' 
                
                # Si no había datos de argumento, asigna '-' por defecto     
                if not argumento_encontrado:
                    argumento = '-'
                    
                # Si no había datos de guion, asigna '-' por defecto     
                if not guion_encontrado:
                    datos_guion = '-'
                    
                # Si no había datos de direccion, asigna '-' por defecto     
                if not direccion_encontrada:
                    datos_direccion = '-'
                
                # Si encontró datos del reparto en el resumen de la película, intenta extraerlos de la página
                if reparto_encontrado:
                    
                    try:
                        
                        if 'reparto' in driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div[1]').text.lower():
                            
                            # Extrae los datos del reparto principal
                            datos_reparto = driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div[2]').text
                            
                            # Separa los datos obtenidos y se queda sólo con los nombres de los actores
                            datos_reparto = datos_reparto.split('\n')[::2] 
                            
                            # Si la longitud de la lista de actores obtenida es superior a 10, la limitamos 
                            if len(datos_reparto) > 10:
                                datos_reparto = datos_reparto[:10]
                        
                        elif 'reparto' in driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[1]').text.lower():
                            
                            # Extrae los datos del reparto principal
                            datos_reparto = driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]').text
                            
                            # Separa los datos obtenidos y se queda sólo con los nombres de los actores
                            datos_reparto = datos_reparto.split('\n')[::2] 
                            
                            # Si la longitud de la lista de actores obtenida es superior a 10, la limitamos 
                            if len(datos_reparto) > 10:
                                datos_reparto = datos_reparto[:10]  
                                
                        elif 'reparto' in driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[1]').text.lower():
                            
                            # Extrae los datos del reparto principal
                            datos_reparto = driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]').text
                            
                            # Separa los datos obtenidos y se queda sólo con los nombres de los actores
                            datos_reparto = datos_reparto.split('\n')[::2] 
                            
                            # Si la longitud de la lista de actores obtenida es superior a 10, la limitamos 
                            if len(datos_reparto) > 10:
                                datos_reparto = datos_reparto[:10]       
                    
                    except:
                        datos_reparto = '-'
                        
                # Cambia la variable a positiva, ya que ha encontrado la película en IMDB
                encontrada = True
                break
        
        # Si encontró la película en IMDB:
        if encontrada:
            # Crea una tupla con todos los detalles de la película
            tupla_peli = (puntuacion , datos_direccion , datos_guion , argumento , duracion_minutos , nombre_peli , anio_peli)
            
            # Añade la tupla al listado de películas
            lista_detalles_peliculas.append(tupla_peli)
            
            # Añade el listado de actores a la lista en caso de haberlos encontrado
            if reparto_encontrado and datos_reparto != '-':
                lista_actores.extend(datos_reparto)
        
        # Si no la encontró, crea una tupla con resultados vacíos y la peli    
        else:
            tupla_peli = (np.nan , np.nan , np.nan , np.nan , np.nan , nombre_peli , anio_peli)
            
            # Añade la tupla al listado de películas
            lista_detalles_peliculas.append(tupla_peli)
        
        sleep(random.randint(1 , 3) + random.uniform(0.1 , 0.5))  
        
        if indice % 100 == 0:    
            df_lista_detalles_peliculas = pd.DataFrame(lista_detalles_peliculas)
            df_lista_detalles_peliculas.to_csv(salida_detalles)
            
            df_lista_actores = pd.DataFrame(list(set(lista_actores)))
            df_lista_actores.to_csv(salida_actores)
    
    # Convertimos la lista de actores a set para que elimine los duplicados y luego volvemos a pasarlo a lista
    lista_actores = list(set(lista_actores))
    
    #! ------------------------------------------------------------------------------ Guarda los archivos
    
    df_lista_detalles_peliculas = pd.DataFrame(lista_detalles_peliculas)
    df_lista_detalles_peliculas.to_csv(salida_detalles)
    
    df_lista_actores = pd.DataFrame(lista_actores)
    df_lista_actores.to_csv(salida_actores)
    
    return lista_detalles_peliculas , lista_actores   

In [25]:
#! Los dos últimos elementos que se le dan a la función son la ruta o nombre donde queremos que se guarden los detalles de la película y la ruta o nombre donde queremos que se guarden los actores 

detalles_pelis , lista_actores = extraccion_detalles_imdb('Drama/pelis_drama_2005_2009.csv', 'nombre' , 'año estreno' , 'detalles_pelis_drama_2005_2009_2.csv' , 'actores_pelis_drama_2005_2009_2.csv')

# SI DA ERROR: 
# ------------ Si el archivo no se encuentra en la misma ubicación que este jupyter, tendremos que darle la ruta. P.e: en lugar de 'pelis_drama_2000_2004.csv' >>> 'Drama/pelis_drama_2000_2004.csv'
# ------------ Hay que sustituir '1' y '2' por el nombre de las columnas en las que aparezca el título de la película y el año
# ------------ Por ejemplo: '1' >>> 'nombre' y '2' >>> 'aÃ±o estreno'
# ------------ Podemos saber el nombre de las columnas abriendo el archivo csv en un excel y mirando en la celda A1

--- Empieza la extracción de datos ---
 
Error al obtener datos del resultado
 
Error al obtener datos del resultado
 
Error al obtener datos del resultado
 
Hago un pequeño break. Estoy en la iteración número 126.
Volveré en unos 5-10 segundos
¡Ya estoy de vuelta! Sigo con lo mío
 
Error al obtener datos del resultado
 
Error al obtener datos del resultado
 
Error al obtener datos del resultado
 
Error al obtener datos del resultado
 
Error al obtener datos del resultado
 
Error al obtener datos del resultado
 
Hago un pequeño break. Estoy en la iteración número 234.
Volveré en unos 5-10 segundos
¡Ya estoy de vuelta! Sigo con lo mío
 
Error al obtener datos del resultado
 
Error al obtener datos del resultado
 
Error al obtener datos del resultado
 
Error al obtener datos del resultado
 
Hago un pequeño break. Estoy en la iteración número 340.
Volveré en unos 5-10 segundos
¡Ya estoy de vuelta! Sigo con lo mío
 
Error al obtener datos del resultado
 
No se encuentra la barra de búsqued